In [1]:
import json
import os
import bz2
import io
from bz2 import BZ2File
import pandas as pd
import numpy as np
import re
from utils import *

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 100)

In [2]:
files_dir = "../data/impresso-data"
archives = os.listdir(files_dir)
date = "1984"

* Extract interesting archives

In [3]:
regex = '(.*198[3-5].*)'
gdl_ = extract_particular_files(archives, regex, 'GDL')      
jdg_ = extract_particular_files(archives, regex, 'JDG') 

* Extract content  and preprocessing

In [4]:
 df_all, df_gdl, df_jdg = run(gdl_, jdg_, date, files_dir)

GDL-1983.jsonl.bz2 : 58449 articles à extraire
GDL-1984.jsonl.bz2 : 62830 articles à extraire
GDL-1985.jsonl.bz2 : 61003 articles à extraire
JDG-1983.jsonl.bz2 : 55278 articles à extraire
JDG-1984.jsonl.bz2 : 57551 articles à extraire
JDG-1985.jsonl.bz2 : 60329 articles à extraire


/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,mag,date,page,ppage,text,length
0,GDL,1983-10-28,1,NaN,La faillite de la politique agricole commune COMMUNAUTÉ EUROPÉENNE Pa r Françoi s Schaller Que c...,2240
1,GDL,1983-10-28,2,NaN,AIR FRANCE LE CLUB ! VOTRE INTERLOCUTEUR A SUR VOUS LAVANTAGE DU TERRAIN. UNE EXCELLENTE RAISON ...,1897
2,GDL,1983-10-28,3,NaN,point de vue 50 km / heure : qui va payer ? A A Contre l'avis unanime des Conseils ^ ^ d'Etat de...,1153
3,GDL,1983-10-28,4,NaN,"LAISSEZ DONC S' A L' TOUJOURS PLUS ALLER VAU- EAU CES BULLETINS. A présent, rien de plus facile ...",1902
4,GDL,1983-10-28,5,NaN,LES BRONZETTIS DE JANDENI Morceaux d'un autre temps Jandeni : Bijou (g) Que renferment-ils en eu...,2065


Length df: 67       494
145      455
91       447
77       405
96       403
        ... 
36470      1
36726      1
13010      1
8300       1
33663      1
Name: length, Length: 11487, dtype: int64


/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,mag,date,page,ppage,text,length
0,GDL,1983-10-28,1,NaN,La faillite de la politique agricole commune COMMUNAUTÉ EUROPÉENNE Pa r Françoi s Schaller Que c...,2240
1,GDL,1983-10-28,2,NaN,AIR FRANCE LE CLUB ! VOTRE INTERLOCUTEUR A SUR VOUS LAVANTAGE DU TERRAIN. UNE EXCELLENTE RAISON ...,1897
2,GDL,1983-10-28,3,NaN,point de vue 50 km / heure : qui va payer ? A A Contre l'avis unanime des Conseils ^ ^ d'Etat de...,1153
3,GDL,1983-10-28,4,NaN,"LAISSEZ DONC S' A L' TOUJOURS PLUS ALLER VAU- EAU CES BULLETINS. A présent, rien de plus facile ...",1902
4,GDL,1983-10-28,5,NaN,LES BRONZETTIS DE JANDENI Morceaux d'un autre temps Jandeni : Bijou (g) Que renferment-ils en eu...,2065


Length df: 67       328
145      301
91       300
77       298
96       274
        ... 
14590      1
35939      1
5653       1
35155      1
32976      1
Name: length, Length: 9129, dtype: int64


/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,mag,date,page,ppage,text,length
0,JDG,1983-10-28,1,NaN,"Le couple, la TV et le « psy » Grande première télévisuelle mercredi soir dernier sur Antenne 2 ...",3649
1,JDG,1983-10-28,2,NaN,"AIR FRANCE LE CLUB ! VOTRE INTERLOCUTEUR A SUR VOUS ~ AVANTAGE DU TERRAIN. UNE', EXCELLENTERAISO...",1884
2,JDG,1983-10-28,4,NaN,"A présent, rien de plus facile que d'oublier toutes les opérations inhérentes au paiement de fac...",1863
3,JDG,1983-10-28,5,NaN,"La France fait obstacle au flnancement du programme cc esprit » Luxembourg, 27 (ATS / AFP).-La F...",1385
4,JDG,1983-10-28,6,NaN,"I DIVERS..''.', "" ., .: ... ........ (!] HL, lesPécioliste, mondial du courrier aérien. Echantil...",954


Length df: 201      182
67       166
145      154
134      147
91       147
        ... 
37859      1
6978       1
9077       1
34570      1
7223       1
Name: length, Length: 9673, dtype: int64


* Keywords filtering

In [5]:
#keywords_1962 =['nucléaire', 'Cuba', 'guerre', 'tension','risque', 'bombe', 'guerre nucléaire']
keywords_1984 = ['nucléaire','initiatives', 'initiative', 'centrale', 'centrales', 'atomique', 'atomiques', 'energie', 'votation']
cleaned_files_dir = "../data/filtered/"
unwanted = ['tableau hebdomadaire']#, '(\/(\s\d*)*\s)*']
archives = os.listdir(files_dir)

In [6]:
#df_all = load_corpus(cleaned_files_dir + "cleaned_all" + date +".json.bz2")
#df_gdl = load_corpus(cleaned_files_dir + "cleaned_gdl" + date +".json.bz2")
#df_jdg = load_corpus(cleaned_files_dir + "cleaned_jdg" + date +".json.bz2")



In [7]:
df_all_ = keywords_filtering(df_all, keywords_1984)
df_all_ = unwanted_filtering(df_all_, unwanted)


===KEYWORDS===


'After keyword filtered: '

(11104, 16)

=====UNWANTED====


'After keyword filtered: '

(11104, 16)

In [8]:
df_gdl_ = keywords_filtering(df_gdl, keywords_1984)
df_gdl_ = unwanted_filtering(df_gdl_, unwanted)

===KEYWORDS===


'After keyword filtered: '

(5527, 16)

=====UNWANTED====


'After keyword filtered: '

(5527, 16)

In [9]:
df_jdg_ = keywords_filtering(df_jdg, keywords_1984)
df_jdg_ = unwanted_filtering(df_jdg_, unwanted)

===KEYWORDS===


'After keyword filtered: '

(5577, 16)

=====UNWANTED====


'After keyword filtered: '

(5577, 16)

* Export iramutek

In [10]:
display(df_all.head())
display(df_gdl.head())
display(df_jdg.head())

,mag,date,page,ppage,text,length,keywords,keyword_nucléaire,keyword_initiatives,keyword_initiative,keyword_centrale,keyword_centrales,keyword_atomique,keyword_atomiques,keyword_energie,keyword_votation
0,GDL,1983-10-28,1,NaN,La faillite de la politique agricole commune COMMUNAUTÉ EUROPÉENNE Pa r Françoi s Schaller Que c...,2240,1,0,0,1,0,0,0,0,0,0
1,GDL,1983-10-28,2,NaN,AIR FRANCE LE CLUB ! VOTRE INTERLOCUTEUR A SUR VOUS LAVANTAGE DU TERRAIN. UNE EXCELLENTE RAISON ...,1897,0,0,0,0,0,0,0,0,0,0
2,GDL,1983-10-28,3,NaN,point de vue 50 km / heure : qui va payer ? A A Contre l'avis unanime des Conseils ^ ^ d'Etat de...,1153,0,0,0,0,0,0,0,0,0,0
3,GDL,1983-10-28,4,NaN,"LAISSEZ DONC S' A L' TOUJOURS PLUS ALLER VAU- EAU CES BULLETINS. A présent, rien de plus facile ...",1902,0,0,0,0,0,0,0,0,0,0
4,GDL,1983-10-28,5,NaN,LES BRONZETTIS DE JANDENI Morceaux d'un autre temps Jandeni : Bijou (g) Que renferment-ils en eu...,2065,0,0,0,0,0,0,0,0,0,0


,mag,date,page,ppage,text,length,keywords,keyword_nucléaire,keyword_initiatives,keyword_initiative,keyword_centrale,keyword_centrales,keyword_atomique,keyword_atomiques,keyword_energie,keyword_votation
0,GDL,1983-10-28,1,NaN,La faillite de la politique agricole commune COMMUNAUTÉ EUROPÉENNE Pa r Françoi s Schaller Que c...,2240,1,0,0,1,0,0,0,0,0,0
1,GDL,1983-10-28,2,NaN,AIR FRANCE LE CLUB ! VOTRE INTERLOCUTEUR A SUR VOUS LAVANTAGE DU TERRAIN. UNE EXCELLENTE RAISON ...,1897,0,0,0,0,0,0,0,0,0,0
2,GDL,1983-10-28,3,NaN,point de vue 50 km / heure : qui va payer ? A A Contre l'avis unanime des Conseils ^ ^ d'Etat de...,1153,0,0,0,0,0,0,0,0,0,0
3,GDL,1983-10-28,4,NaN,"LAISSEZ DONC S' A L' TOUJOURS PLUS ALLER VAU- EAU CES BULLETINS. A présent, rien de plus facile ...",1902,0,0,0,0,0,0,0,0,0,0
4,GDL,1983-10-28,5,NaN,LES BRONZETTIS DE JANDENI Morceaux d'un autre temps Jandeni : Bijou (g) Que renferment-ils en eu...,2065,0,0,0,0,0,0,0,0,0,0


,mag,date,page,ppage,text,length,keywords,keyword_nucléaire,keyword_initiatives,keyword_initiative,keyword_centrale,keyword_centrales,keyword_atomique,keyword_atomiques,keyword_energie,keyword_votation
0,JDG,1983-10-28,1,NaN,"Le couple, la TV et le « psy » Grande première télévisuelle mercredi soir dernier sur Antenne 2 ...",3649,0,0,0,0,0,0,0,0,0,0
1,JDG,1983-10-28,2,NaN,"AIR FRANCE LE CLUB ! VOTRE INTERLOCUTEUR A SUR VOUS ~ AVANTAGE DU TERRAIN. UNE', EXCELLENTERAISO...",1884,0,0,0,0,0,0,0,0,0,0
2,JDG,1983-10-28,4,NaN,"A présent, rien de plus facile que d'oublier toutes les opérations inhérentes au paiement de fac...",1863,0,0,0,0,0,0,0,0,0,0
3,JDG,1983-10-28,5,NaN,"La France fait obstacle au flnancement du programme cc esprit » Luxembourg, 27 (ATS / AFP).-La F...",1385,0,0,0,0,0,0,0,0,0,0
4,JDG,1983-10-28,6,NaN,"I DIVERS..''.', "" ., .: ... ........ (!] HL, lesPécioliste, mondial du courrier aérien. Echantil...",954,0,0,0,0,0,0,0,0,0,0


In [11]:
creating_iramutek_file(df_all_, "../data/iramutek/corpus_all" + date +".txt")
creating_iramutek_file(df_jdg_, "../data/iramutek/corpus_jdg" + date +".txt")
creating_iramutek_file(df_gdl_, "../data/iramutek/corpus_gdl" + date +".txt")